In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
%aiida
from aiida_cp2k.calculations import Cp2kCalculation
from aiida.orm import StructureData, Dict, FolderData, WorkChainNode
from aiida.orm import Str

from aiida.common.exceptions import NotExistent

import ipywidgets as ipw
from IPython.display import display, clear_output, HTML

import matplotlib.pyplot as plt
from pprint import pprint

import nglview

import ase.io
import tempfile

import re
import numpy as np
import scipy.constants

from collections import OrderedDict

# Nudged Elastic Band calculations

In [ ]:
from tempfile import NamedTemporaryFile
from base64 import b64encode
def render_thumbnail(atoms):
    tmp = NamedTemporaryFile()
    ase.io.write(tmp.name, atoms, format='png')
    raw = open(tmp.name, 'rb').read()
    tmp.close()
    return b64encode(raw).decode()

def display_thumbnail(th):
    return '<img width="400px" src="data:image/png;base64,{}" title="">'.format(th)
def html_thumbnail(th):
    return ipw.HTML('<img width="400px" src="data:image/png;base64,{}" title="">'.format(th))


viewer = nglview.NGLWidget()

style = {'description_width': '120px'}
layout = {'width': '70%'}
slider_image_nr = ipw.IntSlider(description='image nr.:',
                              value=1, step=1,
                              min=1, max=2,
                              style=style, layout=layout)


all_ase=[]

def on_image_nr_change(c):
    visualized_ase = all_ase[slider_image_nr.value-1]
    refresh_structure_view(visualized_ase)
slider_image_nr.observe(on_image_nr_change, 'value')    

clear_output()
display(ipw.VBox([slider_image_nr, viewer]))

In [ ]:
def initialize_structure_view():
    global viewer
    
    if hasattr(viewer, "component_0"):
        viewer.component_0.clear_representations()
        viewer.component_0.remove_unitcell()
        cid = viewer.component_0.id
        viewer.remove_component(cid)
        
    
    if len(all_ase)==0:
        return
    atoms = all_ase[0]
    atoms.pbc = [1, 1, 1]


#From Kristjan    
    viewer.add_component(nglview.ASEStructure(atoms), default_representation=False) # adds ball+stick
    #viewer.add_component(nglview.ASEStructure(slab_atoms)) # adds ball+stick
    viewer.add_unitcell()
    viewer.center()
    
    viewer.component_0.add_ball_and_stick(aspectRatio=10.0, opacity=1.0)
    
        
    # Orient camera to look from positive z
    cell_z = atoms.cell[2, 2]
    com = atoms.get_center_of_mass()
    def_orientation = viewer._camera_orientation
    top_z_orientation = [1.0, 0.0, 0.0, 0,
                         0.0, 1.0, 0.0, 0,
                         0.0, 0.0, -np.max([cell_z, 30.0]) , 0,
                         -com[0], -com[1], -com[2], 1]
    viewer._set_camera_orientation(top_z_orientation)
    

In [ ]:
def refresh_structure_view(viz_atoms):
    global viewer
    
    old_camera_orientation = viewer._camera_orientation
    
    if hasattr(viewer, "component_0"):
        viewer.component_0.clear_representations()
        viewer.component_0.remove_unitcell()
        cid = viewer.component_0.id
        viewer.remove_component(cid)
        
#    if len(all_ase)==0:
#        return
    #atoms = all_ase[im]
    viz_atoms.pbc = [1, 1, 1]


#From Kristjan    
    viewer.add_component(nglview.ASEStructure(viz_atoms), default_representation=False) # adds ball+stick
    #viewer.add_component(nglview.ASEStructure(slab_atoms)) # adds ball+stick
    viewer.add_unitcell()
    #viewer.center()
    
    viewer.component_0.add_ball_and_stick(aspectRatio=10.0, opacity=1.0)
    
    viewer._set_camera_orientation(old_camera_orientation)

In [ ]:
def make_replica_html(structure_data_list, energies, distances):
    html = '<table>'
    
    n_col = 4
    for i, (rep, en, dist) in enumerate(zip(structure_data_list, energies, distances)):
        thumbnail = rep.get_extra('thumbnail')
        # The table cell
        if i%n_col == 0:
            html += '<tr>'
        html += '<td><img width="400px" src="data:image/png;base64,{}" title="">'.format(thumbnail)
        # Output some information about the replica...
        html += '<p><b>Nr: </b>{} <br> <b>Energy:</b> {:.6f} eV <br> <b>Dist. to prev:</b> {:.4f} ang</p>'\
                .format(i, en, dist)
        html += '<p>pk: {}</p>'.format(rep.pk)
        # ... and the download link.
        html += '<p><a target="_blank" href="../export_structure.ipynb?uuid={}">View & export</a></p><td>'\
                .format(rep.uuid)
        if i%n_col == n_col-1:
            html += '</tr>'
            
    html += '</tr>'
    html += '</table>'
    return html

In [ ]:
def sorted_opt_rep_keys(keys):
    return sorted([ (int(key.split('_')[2]), key) for key in keys if 'opt_replica' in key])

def process_and_show_neb(c):
    global all_ase
    
    structure_data_list = []
    
    btn_show.disabled = True
    with main_out:
        clear_output()
        
    wc = drop_nebs.value
    
    for i_rep in range(wc.inputs['nreplicas'].value):
        label = "opt_replica_%d" % i_rep
        structure_data_list.append(wc.outputs[label])
        
    energies_array = wc.outputs['replica_energies'].get_array('energies') * 27.211386245
    distances_array = wc.outputs['replica_distances'].get_array('distances') * 0.529177
    
    energies_array = np.array([e_arr - e_arr[0] for e_arr in energies_array])
        
    #### --------------------------------------------------------------
    ## Add thumbnails to replicas if they are not already added
    ## ans store list of ASE structures for the viz
    for rep in structure_data_list:
        the_ase=rep.get_ase()
        all_ase.append(the_ase)
        if not "thumbnail" in rep.extras:
            rep.set_extra("thumbnail", render_thumbnail(the_ase))
    #### --------------------------------------------------------------
    
    replica_html = make_replica_html(structure_data_list, energies_array[-1], distances_array[-1])
    
    barrier_list = [np.max(e_arr) for e_arr in energies_array]
    
    with main_out:
        f, axarr = plt.subplots(1, 2, figsize=(14, 4))
        
        axarr[0].plot(energies_array[-1], 'o-')
        axarr[0].set_ylabel("Energy (eV)")
        axarr[0].set_xlabel("Replica nr")
        axarr[0].set_title("NEB energy profile")
        
        axarr[1].plot(barrier_list, 'o-')
        axarr[1].axhline(barrier_list[-1], linestyle='--', color='lightgray')
        axarr[1].set_ylabel("Barrier (eV)")
        axarr[1].set_xlabel("Iteration nr")
        axarr[1].set_title("NEB convergence")
        
        plt.show()
        
        display(ipw.HTML(replica_html))
        
        print("List of all replica PKs:")
        rep_pk_str = "["
        for struct in structure_data_list:
            rep_pk_str += "%d " % struct.pk
        print(rep_pk_str[:-1] + "]")
    
    
    btn_show.disabled = False
    slider_image_nr.max=len(all_ase)
    initialize_structure_view()

In [ ]:
qb = QueryBuilder()
qb.append(WorkChainNode,
          filters={
              'attributes.process_label': {'==': 'NEBWorkChain'},
              'attributes.process_state': {'==': 'finished'},
          },
          tag='wc',
          project='*'
)
qb.append(StructureData,
          with_incoming='wc'
)
qb.order_by({WorkChainNode: {'id': 'desc'}})
neb_wcs = qb.all()

In [ ]:
sel_options = OrderedDict([("PK %d: " % (neb_wc[0].pk) + neb_wc[0].description, neb_wc[0]) for neb_wc in neb_wcs])

In [ ]:
style = {'description_width': '120px'}
layout = {'width': '70%'}

drop_nebs = ipw.Dropdown(options = sel_options,
                            description = 'NEB: ', layout=layout, style=style)

btn_show = ipw.Button(description="Show")
btn_show.on_click(process_and_show_neb)

main_out = ipw.Output()

display(drop_nebs, btn_show, main_out)